In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Load Titanic dataset
df = pd.read_csv("/kaggle/input/titanic/train.csv")

# Select features and target
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
df = df[features + ['Survived']].dropna()

# Encode categorical variable
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

X = df[features]
y = df['Survived']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(acc * 100, 2), "%")
print("Confusion Matrix:\n", cm)

Accuracy: 74.83 %
Confusion Matrix:
 [[71 16]
 [20 36]]


In [3]:
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

mask = test_df[features].notna().all(axis=1)

valid_x = test_df.loc[mask, features]
valid_pred = model.predict(valid_x).astype(np.int64)
# y_pred = pd.Series(np.nan, index=test_df.index)
y_pred = pd.Series([''] * len(test_df), index=test_df.index)
y_pred[mask] = valid_pred



In [4]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': y_pred
})
submission = submission
submission.to_csv('submission.csv', index=False)
